### Conjuntos
$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$
$$ \text{CAMIONES} = \{1, 2, 3, 4, 5, 6, 7\} $$

### Parámetros
$$ \text{CAPACIDAD\_CAMION}_c = 70 \text{ para cada camión } c \in \text{CAMIONES} $$
$$ \text{DEMANDA}_{cl}: \text{Demanda del cliente } cl \in \text{CLIENTES} $$
$$ \text{DISTANCIAS}_{n1,n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$
$$ \text{TIEMPO\_SERVICIO}_{cl}: \text{Tiempo de servicio en el cliente } cl \in \text{CLIENTES} $$
$$ \text{VENTANA\_TIEMPO}_{cl} = [a_{cl}, b_{cl}]: \text{Ventana de tiempo para el cliente } cl \in \text{CLIENTES} $$
$$ \text{TIEMPO\_TRAVEL}_{n1,n2}: \text{Tiempo de viaje entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$

### Variables de Decisión
$$ x_{n1, n2, c} = \begin{cases} 
1 & \text{Si el camión } c \text{ viaja de } n1 \in \text{NODOS} \text{ a } n2 \in \text{NODOS} \\ 
0 & \text{d.l.c.} 
\end{cases} $$
$$ u_{n, c} \in \mathbb{R}^{+} \text{: Carga acumulada del vehículo } c \text{ al llegar al nodo } n \in \text{NODOS} $$
$$ t_{n, c} \in \mathbb{R}^{+} \text{: Tiempo de llegada del vehículo } c \text{ al nodo } n \in \text{NODOS} $$

### Función Objetivo
$$ \text{Minimizar } \sum_{c \in \text{CAMIONES}} \sum_{n1 \in \text{NODOS}} \sum_{n2 \in \text{NODOS}} \text{DISTANCIAS}_{n1, n2} \cdot x_{n1, n2, c} $$

### Restricciones

1. **Prohibición de bucles:**
$$ \forall n \in \text{NODOS}, \forall c \in \text{CAMIONES}: x_{n, n, c} = 0 $$

2. **Asignación única de salidas para cada cliente:**
$$ \forall cl \in \text{CLIENTES}: \sum_{n \in \text{NODOS}} \sum_{c \in \text{CAMIONES}} x_{cl, n, c} = 1 $$

3. **Asignación única de entradas para cada cliente:**
$$ \forall cl \in \text{CLIENTES}: \sum_{n \in \text{NODOS}} \sum_{c \in \text{CAMIONES}} x_{n, cl, c} = 1 $$

4. **Restricciones para evitar subtours:**
$$ \forall cl1, cl2 \in \text{CLIENTES}, cl1 \neq cl2, \forall c \in \text{CAMIONES}: u_{cl2, c} \geq u_{cl1, c} + \text{DEMANDA}_{cl2} - 10000 \cdot (1 - x_{cl1, cl2, c}) $$

5. **Demanda mínima en cada cliente:**
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}: u_{cl, c} \geq \text{DEMANDA}_{cl} $$

6. **Demanda máxima permitida por el camión:**
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}: u_{cl, c} \leq \text{CAPACIDAD\_CAMION}_{c} $$

7. **Visita de un camión desde el depósito a un cliente:**
$$ \forall c \in \text{CAMIONES}: \sum_{cl \in \text{CLIENTES}} x_{0, cl, c} \leq 1 $$

8. **Retorno de un camión del cliente al depósito:**
$$ \forall c \in \text{CAMIONES}: \sum_{cl \in \text{CLIENTES}} x_{cl, 0, c} \leq 1 $$

9. **Flujo de continuidad:**
$$ \forall c \in \text{CAMIONES}, \forall n \in \text{NODOS}: \sum_{n' \in \text{NODOS}} x_{n', n, c} = \sum_{n' \in \text{NODOS}} x_{n, n', c} $$

10. **Restricciones de ventana de tiempo:**
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}: a_{cl} \leq t_{cl, c} \leq b_{cl} $$

11. **Tiempo de llegada y servicio:**
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}, \forall n \in \text{NODOS}: t_{cl, c} \geq t_{n, c} + \text{TIEMPO\_TRAVEL}_{n, cl} + \text{TIEMPO\_SERVICIO}_{n} - 10000 \cdot (1 - x_{n, cl, c}) $$

12. **Tiempo de llegada del depósito a los clientes:**
$$ \forall cl \in \text{CLIENTES}, \forall c \in \text{CAMIONES}: t_{cl, c} \geq \text{TIEMPO\_TRAVEL}_{0, cl} + \text{TIEMPO\_SERVICIO}_{0} - 10000 \cdot (1 - x_{0, cl, c}) $$


In [ ]:
from docplex.mp.model import Model
import random

# Definición de los conjuntos
N = 10  # Número de clientes (ajusta según sea necesario)
CLIENTES = list(range(1, N + 1))
NODOS = [0] + CLIENTES
CAMIONES = [1, 2, 3, 4, 5, 6, 7]

# Parámetros
CAPACIDAD_CAMION = {c: 70 for c in CAMIONES}
DEMANDAS = {cl: random.randint(10, 20) for cl in CLIENTES}
COORDENADAS = {n: {"x": random.randint(0, N * 10), "y": random.randint(0, N * 10)} for n in NODOS}
DISTANCIAS = {(n1, n2): ((COORDENADAS[n1]["x"] - COORDENADAS[n2]["x"]) ** 2 + (COORDENADAS[n1]["y"] - COORDENADAS[n2]["y"]) ** 2) ** 0.5 for n1 in NODOS for n2 in NODOS}
TIEMPO_SERVICIO = {cl: random.randint(5, 15) for cl in CLIENTES}
VENTANA_TIEMPO = {cl: (random.randint(0, 50), random.randint(51, 100)) for cl in CLIENTES}
TIEMPO_TRAVEL = DISTANCIAS  # Asumimos que el tiempo de viaje es proporcional a la distancia

# Modelo
model = Model("CVRP")

# Variables de decisión
x = model.binary_var_dict([(nodo1, nodo2, c) for nodo1 in NODOS for nodo2 in NODOS for c in CAMIONES], name="x")
u = model.continuous_var_dict([(nodo, c) for nodo in NODOS for c in CAMIONES], name="u")
t = model.continuous_var_dict([(nodo, c) for nodo in NODOS for c in CAMIONES], name="t")

# Función objetivo
model.minimize(model.sum(DISTANCIAS[(nodo1, nodo2)] * x[(nodo1, nodo2, c)] for nodo1 in NODOS for nodo2 in NODOS for c in CAMIONES))

# Restricciones
for nodo in NODOS:
    for c in CAMIONES:
        model.add_constraint(x[nodo, nodo, c] == 0)

for cl in CLIENTES:
    model.add_constraint(model.sum(x[cl, nodo, c] for nodo in NODOS for c in CAMIONES) == 1)

for cl in CLIENTES:
    model.add_constraint(model.sum(x[nodo, cl, c] for nodo in NODOS for c in CAMIONES) == 1)

for cl1 in CLIENTES:
    for cl2 in CLIENTES:
        for c in CAMIONES:
            if cl1 != cl2:
                model.add_constraint(u[cl2, c] >= u[cl1, c] + DEMANDAS[cl2] - 10000 * (1 - x[cl1, cl2, c]))

for cl in CLIENTES:
    for c in CAMIONES:
        model.add_constraint(u[cl, c] >= DEMANDAS[cl])

for cl in CLIENTES:
    for c in CAMIONES:
        model.add_constraint(u[cl, c] <= CAPACIDAD_CAMION[c])

for c in CAMIONES:
    model.add_constraint(model.sum(x[0, cl, c] for cl in CLIENTES) <= 1)
    model.add_constraint(model.sum(x[cl, 0, c] for cl in CLIENTES) <= 1)

for c in CAMIONES:
    for nodo in NODOS:
        model.add_constraint(model.sum(x[nodo_, nodo, c] for nodo_ in NODOS) == model.sum(x[nodo, nodo_, c] for nodo_ in NODOS))

# Restricciones de ventanas de tiempo
for cl in CLIENTES:
    for c in CAMIONES:
        a_cl, b_cl = VENTANA_TIEMPO[cl]
        model.add_constraint(t[cl, c] >= a_cl)
        model.add_constraint(t[cl, c] <= b_cl)

# Tiempo de llegada y servicio
for cl in CLIENTES:
    for c in CAMIONES:
        for n in NODOS:
            if n != cl:
                model.add_constraint(t[cl, c] >= t[n, c] + TIEMPO_TRAVEL[(n, cl)] + TIEMPO_SERVICIO[n] - 10000 * (1 - x[n, cl, c]))

# Tiempo de llegada del depósito a los clientes
for cl in CLIENTES:
    for c in CAMIONES:
        model.add_constraint(t[cl, c] >= TIEMPO_TRAVEL[(0, cl)] + TIEMPO_SERVICIO[0] - 10000 * (1 - x[0, cl, c]))